# Legal Moves

This notebook is being created to address an observation of Purple Penguin which makes the math of quaternion series appear inconsistent. No math is ever completely free. I hope to show with reasonable constraints, quaternion series are not internally inconsistent.

Load the needed libraries.

In [1]:
%%capture
%matplotlib inline
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import math

# To get equations the look like, well, equations, use the following.
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from IPython.display import display

# Tools for manipulating quaternions.
import Q_tools as qt;

from IPython.core.display import display, HTML, Math, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

There are two types of playeres in this notebook: quaternions Q (upper case) and quaternion series qs (lower case). Purple Penguin wrote down the following pair of Quaternions and quaternion series:

In [2]:
q_0 = qt.QH().q_0()
q_1 = qt.QH().q_1()
q_i = qt.QH().q_i()
q_ni = q_i.flip_signs()

A = q_1
B = q_i
a = qt.QHStates([q_1, q_0])
b = qt.QHStates([q_0, q_i])

A.print_state("Quaternion A: ", 1)
B.print_state("Quaternion B: ", 1)
a.print_state("quaternion series a: ", 1)
b.print_state("quaternion series b: ")

Quaternion A: 
(1, 0, 0, 0) 1

Quaternion B: 
(0, 1, 0, 0) i

quaternion series a: 
n=1: (1, 0, 0, 0) 1
n=2: (0, 0, 0, 0) 0
sum= (1, 0, 0, 0) 1+0

quaternion series b: 
n=1: (0, 0, 0, 0) 0
n=2: (0, 1, 0, 0) i
sum= (0, 1, 0, 0) 0+i


Notice that the sum of the states in quaternion series a equals the Quaternion A, while the sum of quaternion series b equals Quaternion B.

Purple Penguin then made the following observation of about the products of the quaternions and quaternion series:

In [3]:
AB = A.Euclidean_product(B)
ab = a.Euclidean_product("bra", ket=b)
AB.print_state("Quaternion Euclidean product AB: ", 1)
ab.print_state("Quaternion dot product ab: ")

Quaternion Euclidean product AB: 
(0, 1, 0, 0) 1*xi

Quaternion dot product ab: 
(0, 0, 0, 0) 1*x0+0*xi


Since these are clearly different, the math of quaternion series is not logically consistent.

There are two group operations one can do on quaternions and quaternion series: addition and multiplication. Purple penguin imposed a constraint on addition, namely that the sum of of the states in the quaternion series must be equal to the quaternion. There was no similar constraint on multiplication. It fact, it is easy to impose a reasonable constraint, namely that $AB=<a|b>$.

How would this be done in practice? What would be the analog in more traditional vector spaces? In vector spaces one has the basis vector and a real or complex number that is the magnitude. One cannot add vectors that have different basis vectors. Only if the vectors have exactly the same basis is one allowed to add them together.

I can see two ways to fix this issue. The first way is to start with quaternion series b that works for both addition and multiplication.

In [4]:
b_ok = qt.QHStates([q_i, q_0])
ab_ok = a.product("bra", ket=b_ok)

ab_ok.print_state("<a|b_ok>")
print("Does AB=<a|b>?: ", AB.equals(ab_ok))

<a|b_ok>
(0, 1, 0, 0) 1xi+0x0
Does AB=<a|b>?:  True


A more interesting fix is to use a spin matrix to rotate the quaternion series b to look exactly like the quaternion series a, analogous to choosing the same basis vector.

In [5]:
sigma_4_b = qt.QHStates([q_0, q_ni, q_0, q_0])
b_rotated = b.Euclidean_product("ket", operator=sigma_4_b)

sigma_4_b.print_state("sigma_4_b:", 1)
b_rotated.print_state("b_rotated:")

sigma_4_b:
n=1: (0, 0, 0, 0) 0
n=2: (0, -1, 0, 0) -i
n=3: (0, 0, 0, 0) 0
n=4: (0, 0, 0, 0) 0
sum= (0, -1, 0, 0) 0+-i+0+0

b_rotated:
n=1: (1, 0, 0, 0) 0x0+-ixi
n=2: (0, 0, 0, 0) 0x0+0xi
sum= (1, 0, 0, 0) 0x0+-ixi+0x0+0xi


Now the constraint on addition is violated, so fix that with another operator:

In [6]:
b_rotated_i = b_rotated.product("ket", operator=qt.QHStates([q_i]))

b_rotated_i.print_state("b_rotated_i", 1)
print("Is b_ok = b_rotated_i?: ", b_ok.equals(b_rotated_i))

b_rotated_i
n=1: (0, 1, 0, 0) ix0x0+-ixi+0x0x0+0xi
n=2: (0, 0, 0, 0) 0x0x0+-ixi+ix0x0+0xi
sum= (0, 1, 0, 0) ix0x0+-ixi+0x0x0+0xi+0x0x0+-ixi+ix0x0+0xi

Is b_ok = b_rotated_i?:  True


Since the two operators sigma_a and i acting on quaternion series b recreate the quaternion series b_ok which passed the multiplication criteria, this is good.

Confirm that one could instead decide to work with quaternion series a to pass the multiplication criteria.

In [7]:
a_ok = qt.QHStates([q_0, q_1])
a_okb = a_ok.Euclidean_product("bra", ket=b)

a_ok.print_state("quaternion series a_ok", 1)
a_okb.print_state("<a_ok|b>")
print("AB=<a_ok|b>?: ", AB.equals(a_okb))

quaternion series a_ok
n=1: (0, 0, 0, 0) 0
n=2: (1, 0, 0, 0) 1
sum= (1, 0, 0, 0) 0+1

<a_ok|b>
(0, 1, 0, 0) 0*x0+1*xi
AB=<a_ok|b>?:  True


In [8]:
sigma_4_a = qt.QHStates([q_0, q_1, q_0, q_0])
a_rotated = a.Euclidean_product("bra", operator=sigma_4_a)
a_rotatedb = a_rotated.Euclidean_product("bra", ket=b)

a_rotated.print_state("a_rotated", 1)
a_rotatedb.print_state("<a_rotated|b>", 1)
print("AB=<a_rotated|b>?: ", AB.equals(a_rotatedb))

a_rotated
n=1: (0, 0, 0, 0) 1*x0+0*x0
n=2: (1, 0, 0, 0) 1*x1+0*x0
sum= (1, 0, 0, 0) 1*x0+0*x0+1*x1+0*x0

<a_rotated|b>
(0, 1, 0, 0) 1*x0+0*x0*x0+1*x1+0*x0*xi

AB=<a_rotated|b>?:  True


To create a large collection of quaternions and a faithful collection of quaternion series would be a tricky affair. The simple part is being consistent with the addition rule. The multiplication constraint would for large numbers might become impossible to figure out. The reason is the constrain requires an examination of every pair of numbers to assure that $AB=<a|b>$.

If the question was asked in reverse, if you had a big pile of quaternion series, would it be possible to figure out the relevant quaternions? Sure, use the addition rule. Let's test if this is the case with three quaternion series with three state dimensions.

In [9]:
random_1 = qt.QHStates([qt.QH().q_random(), qt.QH().q_random(), qt.QH().q_random()])
random_2 = qt.QHStates([qt.QH().q_random(), qt.QH().q_random(), qt.QH().q_random()])
random_3 = qt.QHStates([qt.QH().q_random(), qt.QH().q_random(), qt.QH().q_random()])
R_1 = random_1.summation()
R_2 = random_2.summation()
R_3 = random_3.summation()

random_1.print_state("random quaternion series 1: ", 1)
random_2.print_state("random quaternion series 2: ", 1)
random_3.print_state("random quaternion series 3: ", 1)
R_1.print_state("random quaternion series sum 1: ", 1)
R_2.print_state("random quaternion series sum 2: ", 1)
R_3.print_state("random quaternion series sum 3: ", )

random quaternion series 1: 
n=1: (0.42120304156215493, 0.7643585999105484, 0.6878677170238532, 0.5300690478391255) ?
n=2: (0.8901699840622334, 0.3055526281315578, 0.56078097307889, 0.13105323396542257) ?
n=3: (0.5575742637130741, 0.6445482853904667, 0.4551217784990512, 0.2844596137619081) ?
sum= (1.8689472893374623, 1.714459513432573, 1.7037704686017943, 0.9455818955664561) ?+?+?

random quaternion series 2: 
n=1: (0.6310448426498506, 0.7105094013135147, 0.0073229896510117465, 0.3589652421036602) ?
n=2: (0.7929711364079036, 0.10939653638319757, 0.6735353244523724, 0.34240308579252465) ?
n=3: (0.46434838728280414, 0.6661951152763959, 0.2770940747924663, 0.3815349307741446) ?
sum= (1.8883643663405585, 1.4861010529731082, 0.9579523888958504, 1.0829032586703295) ?+?+?

random quaternion series 3: 
n=1: (0.2134658412844862, 0.08109962085940614, 0.9206589523714092, 0.006016514226055203) ?
n=2: (0.7839274339973614, 0.1077530779912208, 0.9998542988629203, 0.8495656721278508) ?
n=3: (0.3543215

Check if the products are all equal.

In [10]:
R_1R_2 = R_1.Euclidean_product(R_2)
random_1random_2 = random_1.Euclidean_product("bra", ket=random_2)
R_1R_2.print_state("R_1R_2", 1)
random_1random_2.print_state("random_1random_2")
print("R_1 R_2=<random_1|random_2>?: ", R_1R_2.equals(random_1random_2))

R_1R_2
(8.733218258521426, -1.399275874784608, -0.9756133775088744, 1.1278904537178238) ?+?+?*x?+?+?

random_1random_2
(3.0890263765886896, -0.6974362865514272, -0.28849104416774213, 0.561506617846943) ?*x?+?*x?+?*x?
R_1 R_2=<random_1|random_2>?:  False


That hypothesis could not have been more wrong, interesting.

Here's my take home message about the relationship between Quaternions and quaternion series: it's complicated. Nature can be like that.